<span style="display:inline-block;position:relative;font-size:48px;background-color: #d9edab;padding:200px 10px;width:100%;text-align:center">TARGET CUSTOMER</span><br><strong style="display:block;font-size:30px;padding-top:20px">PSO</strong>

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time as time

!pip3 install scikit-learn-intelex

from sklearnex import patch_sklearn
patch_sklearn()

import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [40]:
data = pd.read_csv("https://raw.githubusercontent.com/hsuecu/data_set/main/targetCustomer.csv")
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,26110,56,admin.,married,unknown,no,1933,no,no,telephone,19,nov,44,2,-1,0,unknown,no
1,40576,31,unknown,married,secondary,no,3,no,no,cellular,20,jul,91,2,-1,0,unknown,no
2,15320,27,services,married,secondary,no,891,yes,no,cellular,18,jul,240,1,-1,0,unknown,no
3,43962,57,management,divorced,tertiary,no,3287,no,no,cellular,22,jun,867,1,84,3,success,yes
4,29842,31,technician,married,secondary,no,119,yes,no,cellular,4,feb,380,1,-1,0,unknown,no


In [41]:
if __name__=="__main__":
    for i in data.columns:
        data[i].replace('unknown', np.NAN, inplace=True)

In [42]:
data.drop(['poutcome','day','month'], axis=1, inplace=True)
data['contact'].fillna('cellular', inplace=True)
data = data.drop_duplicates(subset=['ID'], keep='first')

In [43]:
if __name__=="__main__":
    count = 0
    for i in range(data.shape[0]):
        for j in data.iloc[i].isna():
            if j:
                count += 1
                break
    print(f'Useless row entries are:{round(count/data.shape[0] * 100, 3)}%')

Useless row entries are:33.149%


# Data Processing

In [44]:
data.dropna(axis=0, inplace=True)

In [45]:
temp={
    'entrepreneur':10,
    'management':9,
    'admin.':8,
    'services':7,
    'technician':6,
    'retired':5,
    'blue-collar':4,
    'self-employed':3,
    'student':2,
    'housemaid':1,
    'unemployed':0
}
data['job']=data['job'].map(temp)
data['subscribed']=data['subscribed'].apply(lambda i: 1 if i=='yes' else 0)

to_oneHotEncode=['marital', 'education',
                 'default', 'housing',
                 'loan', 'contact']
data_intermediate=pd.get_dummies(data=data[to_oneHotEncode],
                                drop_first=False)
data_intermediate.info(verbose=False)

data = pd.concat([data, data_intermediate], axis = 1)
data.drop(to_oneHotEncode, axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30224 entries, 2 to 31646
Columns: 14 entries, marital_divorced to contact_telephone
dtypes: uint8(14)
memory usage: 649.3 KB


In [46]:
X=data.drop(['subscribed', 'ID'], axis=1)
y=data['subscribed']

from imblearn.under_sampling import RandomUnderSampler
cc=RandomUnderSampler(sampling_strategy=1)
X, y=cc.fit_resample(X, y)
y.value_counts()

0    3527
1    3527
Name: subscribed, dtype: int64

In [47]:
from sklearn.model_selection import train_test_split
X_train,  X_test,y_train, y_test = train_test_split(X,y, train_size = 0.7 )

In [48]:
print(f'X_train:{X_train.shape}\n\
X_test:{X_test.shape}\n\
y_train:{y_train.shape}\n\
y_test:{y_test.shape}')

X_train:(4937, 21)
X_test:(2117, 21)
y_train:(4937,)
y_test:(2117,)


In [49]:
# function to evaluate
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, average_precision_score
from sklearn.model_selection import StratifiedKFold
def analyse(model, xtrain, ytrain, xtest, ytest):
    ytrain_pred = model.predict(xtrain)
    ytest_pred = model.predict(xtest)
    try:
        y_score_test = model.decision_function(xtest)
        y_score_train = model.decision_function(xtrain)
    except:
        y_score_test = model.predict_proba(xtest)[:,1]
        y_score_train = model.predict_proba(xtrain)[:,1]

    ret = {'train':{'pr auc': average_precision_score(ytrain, y_score_train),
                'accuracy': accuracy_score(ytrain, ytrain_pred),
                'recall':recall_score(ytrain, ytrain_pred),
                'precision':precision_score(ytrain, ytrain_pred),
                'roc auc':roc_auc_score(ytrain, y_score_train),
                'f1 score': f1_score(ytrain, ytrain_pred)},
           'test':{'pr auc': average_precision_score(ytest, y_score_test),
                'accuracy':accuracy_score(ytest, ytest_pred),
                'recall':recall_score(ytest, ytest_pred),
                'precision':precision_score(ytest, ytest_pred),
                'roc auc':roc_auc_score(ytest, y_score_test),
                'f1 score': f1_score(ytest, ytest_pred)}}
    return ret

In [50]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, auc, precision_recall_curve

def analyseANN(model, xtrain, ytrain, xtest, ytest):
    ypred =  model.predict(xtrain)
    ypred_test = model.predict(xtest)
    precision_train, recall_train, thresholds = precision_recall_curve(ytrain, ypred)
    precision_test, recall_test, thresholds = precision_recall_curve(y_test, ypred_test)

    for i in range(len(ypred)):
        if ypred[i] <=0.5:
            ypred[i] =0
        else: 
            ypred[i] =1
    for i in range(len(ypred_test)):
        if ypred_test[i] <=0.5:
            ypred_test[i] =0
        else: 
            ypred_test[i] =1
    accuracy = accuracy_score(ytrain, ypred)
    recall = recall_score(ytrain, ypred)
    precision = precision_score(ytrain, ypred)
    roc_auc = roc_auc_score(ytrain, ypred)
    f1score = f1_score(ytrain, ypred)

    ret = {'train':{'pr auc': auc(recall_train, precision_train),
                    'accuracy': accuracy,
                    'recall':recall,
                    'precision':precision,
                    'roc auc':roc_auc,
                    'f1 score': f1score},
               'test':{'pr auc': auc(recall_test, precision_test),
                    'accuracy':accuracy_score(ytest, ypred_test),
                    'recall':recall_score(ytest, ypred_test),
                    'precision':precision_score(ytest, ypred_test),
                    'roc auc':roc_auc_score(ytest, ypred_test),
                    'f1 score': f1_score(ytest, ypred_test)}}
    return ret

# BASE CASE

In [ ]:
##                     ##
## LOGISTIC REGRESSION ##
##                     ##

from sklearn.linear_model import LogisticRegression
import json
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
print(json.dumps(analyse(lr, X, y, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.8412847892500843,
    "accuracy": 0.7846611851431812,
    "recall": 0.779699461298554,
    "precision": 0.7875143184421535,
    "roc auc": 0.8603355426794266,
    "f1 score": 0.7835874056133352
  },
  "test": {
    "pr auc": 0.811960365145672,
    "accuracy": 0.7756258856872933,
    "recall": 0.7641963426371511,
    "precision": 0.775390625,
    "roc auc": 0.8447406436544344,
    "f1 score": 0.7697527872031023
  }
}


In [ ]:
##                     ##
## XG BOOST CLASSIFIER ##
##                     ##

from xgboost.sklearn import XGBClassifier
params = {
            'booster':'gbtree',
            'objective':'binary:logistic',
            'max_depth': 3,
            'learning_rate': 1,
            'n_estimators':100,
            "eval_metric": ['logloss'],
            'random_state': 42
        }         
           
          
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)
xgb_clf.fit(X_train, y_train)
print(json.dumps(analyse(xgb_clf, X, y, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.930655021832607,
    "accuracy": 0.8731216331159626,
    "recall": 0.8760986674227389,
    "precision": 0.870913190529876,
    "roc auc": 0.9390891071662413,
    "f1 score": 0.8734982332155479
  },
  "test": {
    "pr auc": 0.8320500941734622,
    "accuracy": 0.7902692489371752,
    "recall": 0.8017324350336862,
    "precision": 0.7777777777777778,
    "roc auc": 0.8616373314572132,
    "f1 score": 0.7895734597156397
  }
}


In [ ]:
##                          ##
## RANDOM FOREST CLASSIFIER ##
##                          ##


from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier(max_depth = 3)
rforest.fit(X_train, y_train)
print(json.dumps(analyse(rforest, X, y, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.8540692633693702,
    "accuracy": 0.800680464984406,
    "recall": 0.8454777431244684,
    "precision": 0.7759562841530054,
    "roc auc": 0.8723425164647879,
    "f1 score": 0.8092265943012211
  },
  "test": {
    "pr auc": 0.8385594239069548,
    "accuracy": 0.7926310817194142,
    "recall": 0.8402309913378249,
    "precision": 0.7617801047120419,
    "roc auc": 0.8644720465839674,
    "f1 score": 0.7990846681922198
  }
}


In [ ]:
##                        ##
## NAIVE BAYES CLASSIFIER ##
##                        ##


from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print(json.dumps(analyse(gnb, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.7972201948062424,
    "accuracy": 0.6933360340287624,
    "recall": 0.8512861736334405,
    "precision": 0.6492949110974862,
    "roc auc": 0.7964332511859293,
    "f1 score": 0.736695652173913
  },
  "test": {
    "pr auc": 0.7496357802788758,
    "accuracy": 0.6863486065186585,
    "recall": 0.8334937439846006,
    "precision": 0.6381724392041267,
    "roc auc": 0.7682885998917898,
    "f1 score": 0.7228714524207013
  }
}


In [ ]:
##                ##
## SVC CLASSIFIER ##
##                ##


from sklearn.svm import SVC
svc = SVC(C=1, kernel='rbf', gamma='scale')
svc.fit(X_train, y_train)
print(json.dumps(analyse(svc, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.8187871807768456,
    "accuracy": 0.7314158395786915,
    "recall": 0.6579581993569131,
    "precision": 0.775094696969697,
    "roc auc": 0.8255994637879626,
    "f1 score": 0.7117391304347825
  },
  "test": {
    "pr auc": 0.7785507930426454,
    "accuracy": 0.7175247992442135,
    "recall": 0.6458132820019249,
    "precision": 0.7447280799112098,
    "roc auc": 0.8075259677762084,
    "f1 score": 0.6917525773195876
  }
}


In [ ]:
##                ##
## ANN CLASSIFIER ##
##                ##


import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

def Model():
    ann = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=10, activation='relu',solver='adam')
    return ann

In [ ]:
_start = time.time()

model= Model()

model.fit(X_train, y_train)

ret = analyseANN(model, X_train, y_train, X_test, y_test)

_end = time.time()

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')
print(json.dumps(ret, indent=2))

running for: 0m, 0s
{
  "train": {
    "pr auc": 0.7791853242639525,
    "accuracy": 0.6834109783269192,
    "recall": 0.4795016077170418,
    "precision": 0.8165639972621492,
    "roc auc": 0.6850345931602767,
    "f1 score": 0.60420359584705
  },
  "test": {
    "pr auc": 0.7492301482605521,
    "accuracy": 0.6702881435994331,
    "recall": 0.46968238691049086,
    "precision": 0.768503937007874,
    "roc auc": 0.6666593752734272,
    "f1 score": 0.5830346475507766
  }
}


# PSO OPTIMIZED

In [51]:
!pip install sklearn-genetic-opt


from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.plots import plot_fitness_evolution, plot_search_space
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
##                     ##
## LOGISTIC REGRESSION ##
##                     ##

from sklearn.linear_model import LogisticRegression
import json

param_grid = {
    'max_iter': Integer(100, 1000),
    'penalty': Categorical(['l1', 'l2']),
    'C': Continuous(1e-6, 1e-2, distribution="log-uniform"),
    'l1_ratio': Continuous(1e-9, 1, distribution='log-uniform')
}


lr = LogisticRegression(solver="saga")
cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=lr, cv=cv,
                               scoring='accuracy', population_size=10,
                               generations=10, tournament_size=3,
                               elitism=True, crossover_probability=0.8,
                               mutation_probability=0.1, param_grid=param_grid,
                               criteria='max', algorithm='eaMuPlusLambda',
                               n_jobs=-1, verbose=True, keep_top_k=4)

In [ ]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.560986	0.0368149  	0.616564   	0.498885   
1  	14    	0.584176	0.0278508  	0.618992   	0.542433   
2  	19    	0.607633	0.00875725 	0.618992   	0.59713    
3  	19    	0.614698	0.00662183 	0.623242   	0.600764   
4  	17    	0.618432	0.00281843 	0.623242   	0.611904   
5  	20    	0.61272 	0.00839237 	0.623242   	0.600965   
6  	17    	0.617175	0.00485814 	0.623242   	0.605016   
7  	19    	0.615347	0.00874457 	0.624459   	0.596512   
8  	18    	0.617876	0.00645558 	0.624459   	0.600358   
9  	17    	0.621202	0.00134671 	0.624459   	0.619219   
10 	15    	0.625678	0.00769567 	0.646741   	0.619828   
running for: 5m, 32s


In [ ]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.808838240717275,
    "accuracy": 0.6489771116062386,
    "recall": 0.9421221864951769,
    "precision": 0.5959827103991864,
    "roc auc": 0.8052958816447162,
    "f1 score": 0.7301043451175828
  },
  "test": {
    "pr auc": 0.7682347432290411,
    "accuracy": 0.6343882853094001,
    "recall": 0.9412897016361886,
    "precision": 0.5783560023654643,
    "roc auc": 0.7874445779711832,
    "f1 score": 0.7164835164835166
  }
}
{'max_iter': 873, 'penalty': 'l2', 'C': 2.4477644422949932e-05, 'l1_ratio': 0.006808570693715838}


In [ ]:
##                          ##
## RANDOM FOREST CLASSIFIER ##
##                          ##
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

param_grid = {
    'n_estimators': Integer(10, 100),
    'max_depth':Integer(3, 10),
    'max_leaf_nodes':Integer(1, 100),
    'max_features':Categorical(['sqrt', 'log2', None]),
    'criterion':Categorical(['gini', 'entropy']),
    'min_weight_fraction_leaf':Continuous(0.01, 0.5, distribution='log-uniform')
}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=rf, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [ ]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.757465	0.0371678  	0.80879    	0.72007    
1  	15    	0.785153	0.0265068  	0.80879    	0.721893   
2  	17    	0.796273	0.0253579  	0.808389   	0.720682   
3  	16    	0.805448	0.00231528 	0.808588   	0.800889   
4  	18    	0.806643	0.00124566 	0.808588   	0.803929   
5  	17    	0.807859	0.00131056 	0.8094     	0.805753   
6  	16    	0.808852	0.000869182	0.810005   	0.807171   
7  	18    	0.809095	0.000603081	0.8094     	0.807372   
8  	20    	0.808649	0.00143869 	0.810613   	0.805548   
9  	20    	0.809155	0.00129048 	0.811829   	0.806156   
10 	19    	0.809703	0.00115139 	0.811832   	0.808182   
running for: 1m, 17s


In [ ]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.8784437048118043,
    "accuracy": 0.8189183714806563,
    "recall": 0.8428456591639871,
    "precision": 0.8065384615384615,
    "roc auc": 0.8927259173965619,
    "f1 score": 0.8242924528301887
  },
  "test": {
    "pr auc": 0.8471652798396443,
    "accuracy": 0.7949929145016533,
    "recall": 0.8277189605389798,
    "precision": 0.7713004484304933,
    "roc auc": 0.8727636999326811,
    "f1 score": 0.798514391829155
  }
}
{'n_estimators': 65, 'max_depth': 7, 'max_leaf_nodes': 73, 'max_features': 'log2', 'criterion': 'gini', 'min_weight_fraction_leaf': 0.02233850270946316}


In [ ]:
##                     ##
## XG BOOST CLASSIFIER ##
##                     ##

from xgboost.sklearn import XGBClassifier

param_grid = {
    'n_estimators': Integer(10, 20),
    'max_depth': Integer(3, 9),
    'gamma': Continuous(1e-6, 1, distribution='log-uniform'),
    'learning_rate': Continuous(1e-6, 1e-2, distribution='log-uniform')
}


xgb = XGBClassifier();

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=xgb, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [ ]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.786915	0.00995324 	0.798661   	0.760785   
1  	19    	0.795787	0.00452023 	0.803523   	0.790563   
2  	17    	0.801682	0.00416929 	0.805752   	0.795423   
3  	17    	0.802937	0.00282988 	0.805752   	0.795426   
4  	19    	0.803929	0.00109112 	0.805752   	0.801904   
5  	17    	0.803545	0.00281723 	0.805348   	0.795424   
6  	19    	0.803828	0.0023267  	0.805348   	0.797446   
7  	17    	0.804963	0.000643904	0.805548   	0.803724   
8  	20    	0.805024	0.000702435	0.805548   	0.803525   
9  	19    	0.802228	0.00392069 	0.805348   	0.796231   
10 	19    	0.802552	0.00369902 	0.805348   	0.794408   
running for: 0m, 44s


In [ ]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.8767150993732609,
    "accuracy": 0.8266153534535143,
    "recall": 0.8227491961414791,
    "precision": 0.8314378554021121,
    "roc auc": 0.8961516709359684,
    "f1 score": 0.8270707070707071
  },
  "test": {
    "pr auc": 0.8240744358152108,
    "accuracy": 0.7888521492678319,
    "recall": 0.7863330125120308,
    "precision": 0.7840690978886756,
    "roc auc": 0.8629221047067878,
    "f1 score": 0.7851994233541567
  }
}
{'n_estimators': 10, 'max_depth': 6, 'gamma': 1.443329674152842e-06, 'learning_rate': 0.00382769285401596}


In [ ]:
##                        ##
## NAIVE BAYES CLASSIFIER ##
##                        ##


from sklearn.naive_bayes import GaussianNB
import json
param_grid = {
    'var_smoothing': Continuous(1e-9, 1, distribution='log-uniform')
}

gnb = GaussianNB();

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=gnb, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [ ]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.663541	0.0753574  	0.719468   	0.515698   
1  	17    	0.713206	0.0213285  	0.733847   	0.66761    
2  	17    	0.729107	0.0068175  	0.734858   	0.713396   
3  	17    	0.732752	0.00309461 	0.734858   	0.725135   
4  	18    	0.735811	0.00271451 	0.740325   	0.730404   
5  	19    	0.737815	0.00379971 	0.740327   	0.728381   
6  	18    	0.737289	0.00361909 	0.740327   	0.730403   
7  	19    	0.73812 	0.00387119 	0.743978   	0.731212   
8  	19    	0.736723	0.0039052  	0.743978   	0.729391   
9  	20    	0.7396  	0.00259627 	0.743978   	0.736078   
10 	17    	0.739903	0.00431274 	0.743978   	0.729395   
running for: 0m, 9s


In [ ]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.7956423555910976,
    "accuracy": 0.7405306866518129,
    "recall": 0.7632636655948553,
    "precision": 0.7329216518718642,
    "roc auc": 0.800282515732519,
    "f1 score": 0.7477849970466628
  },
  "test": {
    "pr auc": 0.7460583498185194,
    "accuracy": 0.7137458667926311,
    "recall": 0.7526467757459095,
    "precision": 0.6914235190097259,
    "roc auc": 0.7668114231430607,
    "f1 score": 0.7207373271889401
  }
}
{'var_smoothing': 1.1045008051867968e-07}


In [ ]:
##                ##
## SVC CLASSIFIER ##
##                ##


from sklearn.svm import SVC

param_grid = {
    'C': Continuous(1e-4, 10, distribution='log-uniform'),
    'gamma': Continuous(1e-4, 10, distribution='log-uniform')
}

svc = SVC()

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=svc, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [ ]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.551833	0.0806192  	0.709541   	0.50395    
1  	16    	0.611464	0.0738838  	0.705085   	0.50395    
2  	18    	0.700466	0.00785945 	0.705085   	0.678752   
3  	19    	0.705267	0.00280877 	0.711769   	0.701638   
4  	18    	0.708224	0.0023763  	0.711769   	0.704681   
5  	17    	0.710169	0.00227443 	0.712176   	0.705085   
6  	19    	0.712275	0.00435298 	0.721086   	0.705489   
7  	15    	0.71827 	0.00427857 	0.722705   	0.711769   
8  	19    	0.720012	0.00158811 	0.721289   	0.71683    
9  	17    	0.71993 	0.000985249	0.721289   	0.718247   
10 	18    	0.721957	0.00289332 	0.727567   	0.718855   
running for: 7m, 47s


In [ ]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.9811722969715477,
    "accuracy": 0.9266761191006684,
    "recall": 0.8991157556270096,
    "precision": 0.952725724020443,
    "roc auc": 0.9806238585471595,
    "f1 score": 0.9251447477253928
  },
  "test": {
    "pr auc": 0.7096823206186883,
    "accuracy": 0.7161076995748701,
    "recall": 0.7507218479307026,
    "precision": 0.6951871657754011,
    "roc auc": 0.7626749711171545,
    "f1 score": 0.7218880148079594
  }
}
{'C': 2.8028038140437936, 'gamma': 0.0005144645648783607}


In [52]:
##                ##
## ANN CLASSIFIER ##
##                ##

import random

class foo(Categorical):
    def __init__(self):
        pass
    def sample(self):
        hidden_layer_count = random.randint(1, 4)
        l = random.sample(range(8, 32), hidden_layer_count)
        return tuple(l)

param_grid = {
    'alpha': Continuous(1e-4, 0.05, distribution='log-uniform'),
    'activation': Categorical(['tanh', 'relu', 'identity', 'logistic']),
    'solver': Categorical(['lbfgs', 'sgd', 'adam']),
    'batch_size': Integer(100, 1000),
    'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),
    'hidden_layer_sizes': foo(),
    'max_iter': Integer(50, 150),
}

ann = Model()

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=ann, cv=cv, scoring='accuracy',
                               population_size=20, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [53]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.671361	0.0838258  	0.759573   	0.511846   
1  	36    	0.726442	0.0242872  	0.753497   	0.649578   
2  	33    	0.7404  	0.0100762  	0.753497   	0.721289   
3  	38    	0.74836 	0.0119732  	0.786512   	0.732026   
4  	34    	0.761182	0.0164302  	0.786512   	0.732026   
5  	38    	0.767106	0.0129671  	0.786512   	0.744179   
6  	36    	0.774671	0.00921605 	0.788333   	0.753293   
7  	35    	0.783189	0.0056521  	0.791576   	0.775167   
8  	37    	0.78557 	0.0050711  	0.791576   	0.776586   
9  	37    	0.787443	0.00334802 	0.79218    	0.781448   
10 	38    	0.787493	0.00339133 	0.79218    	0.779423   
running for: 17m, 6s


In [54]:
import json
print(json.dumps(analyseANN(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.8453095522313565,
    "accuracy": 0.8023090945918574,
    "recall": 0.838682784493539,
    "precision": 0.7735486351403307,
    "roc auc": 0.803305143231797,
    "f1 score": 0.8048000000000001
  },
  "test": {
    "pr auc": 0.8643271959182169,
    "accuracy": 0.8044402456306093,
    "recall": 0.8306737588652482,
    "precision": 0.8077586206896552,
    "roc auc": 0.8025967378754957,
    "f1 score": 0.8190559440559442
  }
}
{'alpha': 0.019230428478922434, 'activation': 'logistic', 'solver': 'adam', 'batch_size': 236, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (30,), 'max_iter': 136}
